In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

/Users/adityasingh/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# reading the furnished dataset
data = pd.read_csv("./data/data_clean_in_text.csv")
data.head()

,textID,text,selected_text,sentiment,label
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,-1
2,088c60f138,my boss is bullying me...,bullying me,negative,-1
3,9642c003ef,what interview! leave me alone,leave me alone,negative,-1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,-1


## Doc2Vec model

In this we will be using Doc2Vec to generate embeddings than rather than using them for recommendation, we will use them for classification using majority vote by the top n similar documents.

### Doc2Vec model for all 3 sentiments

In [3]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from gensim.models.doc2vec import Doc2Vec
from sklearn.metrics import accuracy_score

In [4]:
# Create TaggedDocument objects
tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i,_d in enumerate(data['text'])]

In [5]:
tagged_data[184]

TaggedDocument(words=['finally', 'got', 'a', 'call', 'for', 'marriage', 'counseling', '3', 'days', 'late', '....'], tags=['184'])

In [6]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=0, epochs=100)

In [7]:
model.build_vocab(tagged_data)

In [8]:
model.train(tagged_data, total_examples=model.corpus_count, epochs=100)

In [9]:
print(data.iloc[7722].text)
print(data.iloc[7722].label)

Happy Star Wars Day. MAY THE 4TH BE WITH YOU
1


In [10]:
similar_doc = model.docvecs.most_similar("7722", topn=5)

In [11]:
def getAllVote(similar_doc, printData = False):
    vote = {}
    for x,y in similar_doc:
        xx = int(x)
        if printData:
            print(f"Document ID: {xx}, Similarity Score: {y}, Label associated: {data.iloc[xx].label}")
        if data.iloc[xx].label not in vote.keys():
            vote[data.iloc[xx].label] = 1
        else:
            vote[data.iloc[xx].label] = vote[data.iloc[xx].label]+1
    return vote

In [12]:
def getMaxVoteLabel(m):
    max_key = max(m, key=m.get)
    return max_key

In [13]:
vote = getAllVote(similar_doc, printData = True)
print(vote)
print(f"Max vote is: {getMaxVoteLabel(vote)}")

Document ID: 8487, Similarity Score: 0.9678317904472351, Label associated: 1
Document ID: 22933, Similarity Score: 0.9676925539970398, Label associated: 0
Document ID: 17187, Similarity Score: 0.9650979042053223, Label associated: 1
Document ID: 18573, Similarity Score: 0.9643941521644592, Label associated: 1
Document ID: 13594, Similarity Score: 0.9632465839385986, Label associated: 1
{1: 4, 0: 1}
Max vote is: 1


#### Testing the Doc2Vec model for all 3 sentiments

In [14]:
test_data = pd.read_csv("./data/test.csv")

In [15]:
# Define the mapping
sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}

# Map the values and create a new column
test_data['label'] = test_data['sentiment'].map(sentiment_mapping)

In [16]:
test_data

,textID,text,sentiment,label
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,1
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,-1
3,01082688c6,happy bday!,positive,1
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,1
...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,-1
3530,416863ce47,All alone in this old house again. Thanks for...,positive,1
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,-1
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,1


In [17]:
def getLabelByVoting(x):
    text = word_tokenize(x.lower())
    vector = model.infer_vector(text)
    similar_doc_tags = model.docvecs.similar_by_vector(vector, topn=5)
    vote = getAllVote(similar_doc_tags)
    return getMaxVoteLabel(vote)

In [18]:
actual = test_data.label
predictions = test_data.text.apply(getLabelByVoting)

In [19]:
accuracy_score(actual, predictions)

0.4663271080928127

### Doc2Vec model for only positive and negative sentiments

In [20]:
data_pos_neg = data[data.label != 0]

In [21]:
# Create TaggedDocument objects
tagged_data_pos_neg = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i,_d in enumerate(data_pos_neg['text'])]

In [22]:
tagged_data_pos_neg[184]

TaggedDocument(words=['before', 'i', 'get', 'too', 'distracted', ',', 'i', '`', 'd', 'like', 'to', 'thank', 'my', 'new', 'followers', 'for', 'taking', 'the', 'trouble', 'to', 'follow', 'me', '!', 'and', 'to', 'my', 'others', ':', 'feelin', 'the', 'love'], tags=['184'])

In [80]:
model_pos_neg = gensim.models.doc2vec.Doc2Vec(vector_size=30, min_count=0, epochs=100)

In [81]:
model_pos_neg.build_vocab(tagged_data_pos_neg)

In [82]:
model_pos_neg.train(tagged_data_pos_neg, total_examples=model_pos_neg.corpus_count, epochs=100)

In [83]:
print(data_pos_neg.iloc[7722].text)
print(data_pos_neg.iloc[7722].label)

Refusing to accept that the US holiday is over. Heading to Woodies Longboard  Diner in um ... Hove...
-1


In [94]:
similar_doc_pos_neg = model_pos_neg.docvecs.most_similar("7722", topn=1)

In [95]:
def getAllVote_pos_neg(similar_doc, printData = False):
    vote = {}
    for x,y in similar_doc:
        xx = int(x)
        if printData:
            print(f"Document ID: {xx}, Similarity Score: {y}, Label associated: {data_pos_neg.iloc[xx].label}")
        if data_pos_neg.iloc[xx].label not in vote.keys():
            vote[data_pos_neg.iloc[xx].label] = 1
        else:
            vote[data_pos_neg.iloc[xx].label] = vote[data_pos_neg.iloc[xx].label]+1
    return vote

In [96]:
def getMaxVoteLabel_pos_neg(m):
    max_key = max(m, key=m.get)
    return max_key

In [97]:
vote_pos_neg = getAllVote_pos_neg(similar_doc_pos_neg, printData = True)
print(vote_pos_neg)
print(f"Max vote is: {getMaxVoteLabel(vote_pos_neg)}")

Document ID: 13026, Similarity Score: 0.7098854184150696, Label associated: -1
{-1: 1}
Max vote is: -1


#### Testing the Doc2Vec model

In [98]:
test_data_pos_neg = pd.read_csv("./data/test.csv")

In [99]:
# Define the mapping
sentiment_mapping = {'positive': 1, 'negative': -1, 'neutral': 0}

# Map the values and create a new column
test_data_pos_neg['label'] = test_data_pos_neg['sentiment'].map(sentiment_mapping)
test_data_pos_neg = test_data_pos_neg[test_data_pos_neg.label != 0]

In [100]:
test_data_pos_neg

,textID,text,sentiment,label
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,1
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,-1
3,01082688c6,happy bday!,positive,1
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,1
5,726e501993,that`s great!! weee!! visitors!,positive,1
...,...,...,...,...
3529,e5f0e6ef4b,"its at 3 am, im very tired but i can`t sleep ...",negative,-1
3530,416863ce47,All alone in this old house again. Thanks for...,positive,1
3531,6332da480c,I know what you mean. My little dog is sinkin...,negative,-1
3532,df1baec676,_sutra what is your next youtube video gonna b...,positive,1


In [101]:
def getLabelByVoting_pos_neg(x):
    text = word_tokenize(x.lower())
    vector = model.infer_vector(text)
    similar_doc_tags = model.docvecs.similar_by_vector(vector, topn=5)
    vote = getAllVote(similar_doc_tags)
    return getMaxVoteLabel(vote)

In [102]:
actual_pos_neg = test_data_pos_neg.label
predictions_pos_neg = test_data_pos_neg.text.apply(getLabelByVoting)

In [103]:
accuracy_score(actual_pos_neg, predictions_pos_neg)

0.43155893536121676

### Conclusion

As we can see from the result, the accuracy is not too good for both the cases.

Accuracy for all 3 sentiments: `47.198641765704585%`

Accuracy for only positive and negative sentiments: `43.155893536121676%`